In [1]:
# Uploading for a whole day for 1 account

from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi
import json
import pandas as pd
import os
import datetime
import numpy as np
import glob
import time
BigLotsAccount_id="act_271491453638620"
BigLotsDcart10_id="act_2449979771956630"

list_account_id=[BigLotsAccount_id,BigLotsDcart10_id]
# jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token=json.load(open("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/FB_token/lasest_FB_token_jian.json","r"))['latest_FB_token']

In [2]:
list_latest_Dec=glob.glob("./uploaded_file_FB/output_2021-01-04/*.csv")
print(len(list_latest_Dec))
list_latest_Dec_to_add=[x for x in list_latest_Dec if "add" in x]
list_latest_Dec_to_remove=[x for x in list_latest_Dec if "remove" in x]

list_audience_name=["aud_"+os.path.basename(x).replace("users_to_add_","").replace(".csv","") for x in list_latest_Dec_to_add]

print(len(list_latest_Dec_to_add),len(list_latest_Dec_to_remove))

30
15 15


In [3]:
API_VERSION = 'v8.0'
FacebookAdsApi.init(access_token=jian_token, api_version=API_VERSION)

def func_fetch_df_names(bl_fb_acctount_id):
    myAccount = AdAccount(bl_fb_acctount_id)
    custom_audiences = myAccount.get_custom_audiences(fields=['name'])
    df = pd.DataFrame(custom_audiences)
    return df

def remove_ids_from_account(token,aud_id,list_ids_to_remove,fb_schema=CustomAudience.Schema.email_hash):
    FacebookAdsApi.init(access_token=token, api_version='v8.0')
    audience = CustomAudience(aud_id)
    listlen=len(list_ids_to_remove)
    
    chunck_size=1000
    count_removed=0
    for i in range(int(np.ceil(listlen/chunck_size))):
        starti = i*chunck_size
        if (i+1)*chunck_size<listlen:
            endi = (i+1)*chunck_size
        else:
            endi = listlen
        list0 = list_ids_to_remove[starti:endi]
        
        count_removed+=len(list0)
        audience.remove_users(fb_schema, list0)
        time.sleep(2)
    print("%i IDs removed from the audience id: %i"%(count_removed,aud_id))
    
def add_ids_to_account(token,aud_id,list_ids_to_add,fb_schema=CustomAudience.Schema.email_hash):
    FacebookAdsApi.init(access_token=token, api_version='v8.0')
    audience = CustomAudience(aud_id)
    listlen=len(list_ids_to_add)
    
    chunck_size=30000
    count_added=0
    for i in range(int(np.ceil(listlen/chunck_size))):
        starti = i*chunck_size
        if (i+1)*chunck_size<listlen:
            endi = (i+1)*chunck_size
        else:
            endi = listlen
        list0 = list_ids_to_add[starti:endi]
        
        count_added+=len(list0)
        audience.add_users(fb_schema, list0)
    print("%i IDs added to the audience id: %i"%(count_added,aud_id))

In [4]:
# Fetach all currently existing FB audience ID & Name
df_BLAccount_Aud=func_fetch_df_names(BigLotsAccount_id)
df_BLAccount_Aud=df_BLAccount_Aud[df_BLAccount_Aud['name'].isin(list_audience_name)].reset_index()
del df_BLAccount_Aud['index']
print(df_BLAccount_Aud.shape)
df_BLAccount_Aud.head(2)

(15, 2)


,id,name
0,23845696696190265,aud_T_SignUpNoPurchase_2_12
1,23845696687730265,aud_T_SOTF_Lapsed2530_Unknown


In [28]:
'''
list_id_done_BLAccount_Aud=[23845696696190265,23845696687730265,23845696678520265,23845696662080265,
              23845696655840265,23845696639640265,23845696591270265,23845696582480265,
              23845696542970265,23845696529070265,23845696511720265]
'''

# MS BL Account

In [5]:
list_id_done_BLAccount_Aud=[]

In [6]:
df_BLAccount_Aud=df_BLAccount_Aud[~df_BLAccount_Aud['id'].isin([str(x) for x in list_id_done_BLAccount_Aud])]
print(df_BLAccount_Aud.shape)

(15, 2)


In [7]:
i_counter=0
for i, row in df_BLAccount_Aud.iterrows():
    i_counter+=1
    fb_id=int(row['id'])
    seg=row['name'].replace("aud_","")
    print("fb_id",fb_id)
    print(i_counter,"seg",seg)
    
    file_to_add=[x for x in list_latest_Dec_to_add if seg in x]
    file_to_remove=[x for x in list_latest_Dec_to_remove if seg in x]
    if len(file_to_add)==1 and len(file_to_remove)==1:
        file_to_add=file_to_add[0]
        file_to_remove=file_to_remove[0]
    else:
        print("len_file_error")
        
    df_to_add=pd.read_csv(file_to_add,dtype=str,usecols=["email_address_hash"]).drop_duplicates()
    df_to_remove=pd.read_csv(file_to_remove,dtype=str,usecols=["email_address_hash"]).drop_duplicates()
    
    list_emails_to_add=[x for x in df_to_add['email_address_hash'].tolist() if len(x)==64]
    list_emails_to_remove=[x for x in df_to_remove['email_address_hash'].tolist() if len(x)==64]
    
    print(datetime.datetime.now(),"start adding of %s"%seg,fb_id)
    add_ids_to_account(token=jian_token,
                       aud_id=fb_id,
                       list_ids_to_add=list_emails_to_add,
                       fb_schema=CustomAudience.Schema.email_hash)
    
    print(datetime.datetime.now(),"start removing of %s"%seg,fb_id)    
    remove_ids_from_account(token=jian_token,
                            aud_id=fb_id,
                            list_ids_to_remove=list_emails_to_remove,
                            fb_schema=CustomAudience.Schema.email_hash)
    
    print(datetime.datetime.now(),"Done of %s"%seg,fb_id)
    list_id_done_BLAccount_Aud.append(fb_id)

    time.sleep(10)

fb_id 23845696696190265
1 seg T_SignUpNoPurchase_2_12
2021-01-04 14:28:23.690164 start adding of T_SignUpNoPurchase_2_12 23845696696190265
289577 IDs added to the audience id: 23845696696190265
2021-01-04 14:28:40.036138 start removing of T_SignUpNoPurchase_2_12 23845696696190265
200482 IDs removed from the audience id: 23845696696190265
2021-01-04 14:37:35.481900 Done of T_SignUpNoPurchase_2_12 23845696696190265
fb_id 23845696687730265
2 seg T_SOTF_Lapsed2530_Unknown
2021-01-04 14:37:45.699772 start adding of T_SOTF_Lapsed2530_Unknown 23845696687730265
8131 IDs added to the audience id: 23845696687730265
2021-01-04 14:37:46.783759 start removing of T_SOTF_Lapsed2530_Unknown 23845696687730265
19774 IDs removed from the audience id: 23845696687730265
2021-01-04 14:38:39.932847 Done of T_SOTF_Lapsed2530_Unknown 23845696687730265
fb_id 23845696678520265
3 seg T_SOTF_Lapsed1924_Unknown
2021-01-04 14:38:50.403919 start adding of T_SOTF_Lapsed1924_Unknown 23845696678520265
84235 IDs added to

# Dcart Account

In [8]:
df_BLDcart_Aud=func_fetch_df_names(BigLotsDcart10_id)
df_BLDcart_Aud=df_BLDcart_Aud[df_BLDcart_Aud['name'].isin(list_audience_name)].reset_index()
del df_BLDcart_Aud['index']
print(df_BLDcart_Aud.shape)
df_BLDcart_Aud.head(2)

(15, 2)


,id,name
0,23846060364080632,aud_T_SignUpNoPurchase_2_12
1,23846060344990632,aud_T_SOTF_Lapsed2530_Unknown


In [9]:
list_id_done_BLDart_Aud=[]

In [10]:
i_counter=0
for i, row in df_BLDcart_Aud.iterrows():
    i_counter+=1
    fb_id=int(row['id'])
    seg=row['name'].replace("aud_","")
    print("fb_id",fb_id)
    print(i_counter,"seg",seg)
    
    file_to_add=[x for x in list_latest_Dec_to_add if seg in x]
    file_to_remove=[x for x in list_latest_Dec_to_remove if seg in x]
    if len(file_to_add)==1 and len(file_to_remove)==1:
        file_to_add=file_to_add[0]
        file_to_remove=file_to_remove[0]
    else:
        print("len_file_error")
        
    df_to_add=pd.read_csv(file_to_add,dtype=str,usecols=["email_address_hash"]).drop_duplicates()
    df_to_remove=pd.read_csv(file_to_remove,dtype=str,usecols=["email_address_hash"]).drop_duplicates()
    
    list_emails_to_add=[x for x in df_to_add['email_address_hash'].tolist() if len(x)==64]
    list_emails_to_remove=[x for x in df_to_remove['email_address_hash'].tolist() if len(x)==64]
    
    print(datetime.datetime.now(),"start adding of %s"%seg,fb_id)
    add_ids_to_account(token=jian_token,
                       aud_id=fb_id,
                       list_ids_to_add=list_emails_to_add,
                       fb_schema=CustomAudience.Schema.email_hash)
    
    print(datetime.datetime.now(),"start removing of %s"%seg,fb_id)    
    remove_ids_from_account(token=jian_token,
                            aud_id=fb_id,
                            list_ids_to_remove=list_emails_to_remove,
                            fb_schema=CustomAudience.Schema.email_hash)
    
    print(datetime.datetime.now(),"Done of %s"%seg,fb_id)
    list_id_done_BLDart_Aud.append(fb_id)
    time.sleep(10)

fb_id 23846060364080632
1 seg T_SignUpNoPurchase_2_12
2021-01-04 17:32:59.040137 start adding of T_SignUpNoPurchase_2_12 23846060364080632
289577 IDs added to the audience id: 23846060364080632
2021-01-04 17:33:16.795767 start removing of T_SignUpNoPurchase_2_12 23846060364080632
200482 IDs removed from the audience id: 23846060364080632
2021-01-04 17:42:01.920861 Done of T_SignUpNoPurchase_2_12 23846060364080632
fb_id 23846060344990632
2 seg T_SOTF_Lapsed2530_Unknown
2021-01-04 17:42:12.108390 start adding of T_SOTF_Lapsed2530_Unknown 23846060344990632
8131 IDs added to the audience id: 23846060344990632
2021-01-04 17:42:12.792764 start removing of T_SOTF_Lapsed2530_Unknown 23846060344990632
19774 IDs removed from the audience id: 23846060344990632
2021-01-04 17:43:00.946631 Done of T_SOTF_Lapsed2530_Unknown 23846060344990632
fb_id 23846060340760632
3 seg T_SOTF_Lapsed1924_Unknown
2021-01-04 17:43:11.405788 start adding of T_SOTF_Lapsed1924_Unknown 23846060340760632
84235 IDs added to

In [11]:
len(list_id_done_BLDart_Aud),list_id_done_BLDart_Aud


(15,
 [23846060364080632,
  23846060344990632,
  23846060340760632,
  23846060334740632,
  23846060319650632,
  23846060279180632,
  23846060263680632,
  23846060242300632,
  23846060220210632,
  23846060199320632,
  23846060182190632,
  23846060144470632,
  23846060113100632,
  23846060084000632,
  23846060022670632])